# input-cos-zarr

Read ZARR database as XArray from COS, apply function, store result

In [ ]:
!pip install zarr xarray s3fs cftime

In [3]:
import logging
import os
import re
import sys
import s3fs
import xarray as xr

In [ ]:
# access key id
access_key_id = os.environ.get('access_key_id')

# secret access key
secret_access_key = os.environ.get('secret_access_key')

# cos/s3 endpoint
endpoint = os.environ.get('endpoint')

# cos bucket name
bucket_name = os.environ.get('bucket_name')

# path
path = os.environ.get('path','')

# target
target = os.environ.get('target')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.debug('Parameter: ' + parameter)
    exec(parameter)

In [5]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint}
)

s3_path = f"s3://{bucket_name}/{path}"

# open the zarr file with xarray
ds = xr.open_zarr(store=s3fs.S3Map(s3_path, s3=s3))

In [7]:
import time
start_time = time.time()
ds.to_zarr("./my-dataset.zarr")
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.4f} seconds")

In [ ]:
print('Data written successfully')